In [1]:
!pip install openpyxl
!pip install XlsxWriter
!pip install xlrd
!pip install python-dateutil
!pip install pyreadstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 KB 519.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 361.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 344.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import glob
import os
import pyreadstat
import gc

In [3]:
# pd.set_option('display.max_rows', None)

# input_root_path = "../data"
input_root_path = "../input/travel-time"
ats_path = "../input/atstraffic"
weather_path = "../input/weatherstations"
counter_path = "../input/counterstations"
# output_root_path = "../data"
output_root_path = "../working"

# Load Dataset

## Add Initial Data

In [4]:
def load_csv_to_df(files, filename=None, is_excel=False):
    if is_excel:
        df = pd.concat((pd.read_excel(f) for f in files), ignore_index=True)
    else:
        df = pd.concat((pd.read_csv(f, skipinitialspace = True) for f in files), ignore_index=True)
    
    if filename:
        df.to_csv(os.path.join(output_root_path, filename), header = True, index=True)
    return df

In [5]:
# Load data 

tasks_joined_files = glob.glob(os.path.join(input_root_path, "Update Travel Time Archive_All_Data/Update Travel Time Archive/Travel Time Task/TravelTimeTaskArchive*.csv"))
df_tasks_ini = load_csv_to_df(tasks_joined_files)
# df_tasks_ini = load_csv_to_df(tasks_joined_files, filename="all_TravelTimeTaskArchive.csv")

# summary_joined_files = glob.glob(os.path.join(input_root_path, "Update Travel Time Archive_All_Data/Update Travel Time Archive/Travel Time Summary/TravelTimeSummaryTaskArchive*.csv"))
# df_summary = load_csv_to_df(summary_joined_files)

df_tasks_ini

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
1,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
2,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
3,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
4,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
...,...,...,...,...,...,...,...,...
13438804,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438805,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438806,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438807,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06


## Add Nectar Data

In [6]:
def load_nectar_to_df(files, filename=None):
    df_nectar = pd.DataFrame()
    
    for f in files:
        df_temp = pd.read_csv(f, skipinitialspace = True)
        filename = os.path.splitext(f)[0].split('/')
        df_temp.insert(loc=0, column='AssetNumber', value=int(filename[-1]))
        df_temp.insert(loc=1, column='AssetIdentifier', value=filename[-2])
        df_temp.rename(columns = {
            'Task Date': 'CreatedDate', 'Under Construction': 'UnderConstructionTime',
            'Baseline': 'BaselineTime', 'Delay': 'DelayTime', 'Segment Length': 'SegmentLength'
        }, inplace=True)

        df_nectar = pd.concat([df_nectar,
                               df_temp[['AssetNumber', 'AssetIdentifier', 'CreatedDate', 'UnderConstructionTime', 'BaselineTime', 'DelayTime', 'State', 'SegmentLength']]],
                              axis=0, ignore_index=True)
#         df_nectar = pd.concat([df_nectar, df_temp], axis=0)
        
    return df_nectar

In [7]:
nectar_joined_files = glob.glob(os.path.join(input_root_path, "ATSTraffic Dataset/Travel Time/*/*.csv"))
df_nectar = load_nectar_to_df(nectar_joined_files)

df_nectar

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.363
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.363
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.363
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.363
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.363
...,...,...,...,...,...,...,...,...
1555655,51967047367,Spread 5B,6/4/2022 6:50:36 AM,1.52,1.75,0.00,OK,2.610
1555656,51967047367,Spread 5B,6/4/2022 6:48:40 AM,1.52,1.75,0.00,OK,2.610
1555657,51967047367,Spread 5B,6/4/2022 6:47:18 AM,1.52,1.75,0.00,OK,2.610
1555658,51967047367,Spread 5B,6/4/2022 6:44:35 AM,1.50,1.75,0.00,OK,2.610


## Concat both datasets

In [8]:
df_tasks = pd.concat([df_nectar, df_tasks_ini], axis=0, ignore_index=True)
df_tasks

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.363
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.363
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.363
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.363
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.363
...,...,...,...,...,...,...,...,...
14994464,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.00,0.00,0.00,OK,3.060
14994465,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.00,0.00,0.00,OK,3.060
14994466,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.00,0.00,0.00,OK,3.060
14994467,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.00,0.00,0.00,OK,3.060


In [9]:
del df_tasks_ini
del df_nectar

## Find Unique Assets

In [10]:
def get_unique_assets(filename=None):
    df_asset = pd.concat([df_tasks[['AssetNumber', 'AssetIdentifier']], df_summary[['AssetNumber', 'AssetIdentifier']]], axis=0) 
    df_asset_unique = df_asset.drop_duplicates()
    
    if filename:
        df_asset_unique.to_csv(os.path.join(output_root_path, filename), header = True, index=True)

    return df_asset_unique

# df_asset_unique = get_unique_assets()
# df_asset_unique = get_unique_assets(filename='unique_assets.csv')
# df_asset_unique

# Get Assets

In [11]:
# df_assets_all_active = pd.read_excel(os.path.join(ats_path, 'AllWorkZones.xlsx'), sheet_name = ['HaveBeenActivated'])['HaveBeenActivated']
# df_assets_all_active

In [12]:
df_assets_recom = pd.read_excel(os.path.join(ats_path, 'Recommended Assets.xlsx'))
df_assets_recom

,Asset Number,Segment Name,Segment Length,Under Construction,Baseline,Direct of Travel,Spread,Traffic Impact,Details of Right of Way to work within or close,Description of work,Road Type,Traffic Control Setup,Easting,Northing,Most Recent Activation,Cell Coverage
0,62134790681,AP 3-B-12-A-1,2.746,1.67,1.72,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-12-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN
1,10897197432,AP 3-B-12-A-1,2.746,1.68,1.73,SB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-12-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN
2,12768050215,AP 3-B-13-A-1,2.970,1.77,1.92,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-13-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN
3,56678162306,AP 3-B-13-A-1,2.970,1.80,1.92,SB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-13-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN
4,86851077697,AP 3-B-13-A-3,18.000,NaN,NaN,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of an Access Point along Spread 3-4A o...,Multi-Lane Undivided Roadways,SLAT,346092.86128,5.857159e+06,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,34684428742,T3920.0,4.840,0.00,0.00,SB,Spread 5A,High,SMJV is requesting approval to work within the...,This application is for traffic control at the...,Multi-Lane Divided,Lane Closure,660846.97920,5.552540e+06,NaN,NaN
620,69027783597,T4068.3,3.460,0.00,0.00,SB,Spread 5B,High,Freeway posted speed: 120km/h alignment: large...,Lane closure to allow for installation of CRB ...,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN
621,92801985490,T4068.3,3.480,0.00,0.00,NB,Spread 5B,High,Freeway posted speed: 120km/h alignment: large...,Lane closure to allow for installation of CRB ...,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN
622,16764775025,TWS_460042,3.690,2.73,2.58,SB,Reactivation,High,I/W request approval to work within the Minist...,"The eastbound passing lane will be closed, the...",Multi-Lane Undivided Roadways,Lane Closure,356159.86762,5.876918e+06,4/4/2022,No Cell Coverage Area


In [13]:
# # Remove No Cell Coverage Area entries
# df_assets_recom_no_cell = df_assets_recom[df_assets_recom['Cell Coverage'] != 'No Cell Coverage Area']
# df_assets_recom_no_cell

In [14]:
# df_assets_all_active.columns.difference(df_assets_recom.columns)

# Pre-processing & Cleaning

## Filter Valid Traffic Times

In [15]:
# Filter out entries with zero UnderConstructionTime or BaselineTime
df_valid = df_tasks[(df_tasks['UnderConstructionTime'] != 0) & (df_tasks['BaselineTime'] != 0)].reset_index(drop=True)

# df_valid = df_tasks[}|(df_tasks['UnderConstructionTime'] != 0) & (df_tasks['BaselineTime'] != 0)].rename_axis('AllTasksIndex').reset_index()
# df_valid = df_valid.rename_axis('ValidTasksIndex')
# df_valid.index.name = 'ValidTasksIndex'
df_valid

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.363
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.363
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.363
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.363
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.363
...,...,...,...,...,...,...,...,...
5365467,88582155367,Spread 7 - SB,4/17/2021 6:00:11 PM,21.77,23.60,0.00,OK,32.980
5365468,75946582969,Spread 4B - NB,4/17/2021 6:00:12 PM,43.37,45.35,0.00,OK,70.710
5365469,43132175891,Spread 5B - SB,4/17/2021 6:00:12 PM,53.48,53.73,0.00,OK,87.350
5365470,99369489819,Spread 2 - SB,4/17/2021 6:00:12 PM,165.52,169.87,0.00,OK,286.840


## Match work zone datasets
For all the workzones under the "have been activated" tab, creates a separate spreadsheet for each workzone that contains all the travel time information that you can find in the "Update Travel Time Archive_All_Data" spreadsheet. So for each workzone we would have one spreadsheet with all the travel time information.

In [16]:
# def find_workzone_tt_data(df_assets, df_tt, file_name=None):
#     df_tt_wz = pd.DataFrame()
    
#     for i, location_id in enumerate(df_assets['Asset Number']):
#         df_intersect = df_tt[df_tt['AssetNumber'] == location_id]
#         df_tt_wz = df_tt_wz.append(df_intersect)
        
#         if file_name:
#             with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
#                 df_intersect.to_excel(writer, sheet_name=str(location_id))
            
#         if (i+1) % 100 == 0:
#             print(f'{round((i+1)/len(df_assets)*100, 2)} %')
            
#     return df_tt_wz

In [17]:
df_valid = pd.merge(left=df_valid, right=df_assets_recom[
    ['Asset Number', 'Segment Length', 'Direct of Travel', 'Spread', 'Traffic Impact', 'Road Type', 'Traffic Control Setup']],
                    left_on='AssetNumber', right_on='Asset Number', how='inner')
# df_valid = pd.merge(left=df_valid, right=df_assets_recom_no_cell[
#     ['Asset Number', 'Segment Length', 'Direct of Travel', 'Spread', 'Traffic Impact', 'Road Type', 'Traffic Control Setup']],
#                     left_on='AssetNumber', right_on='Asset Number', how='inner')

df_valid.rename(columns = {
    'SegmentLength': 'SegmentLength_TravelTime',
    'Segment Length':'SegmentLength_ASSETS',
    'Direct of Travel': 'TravelDirect',
    'Traffic Impact': 'TrafficImpact',
    'Road Type': 'RoadType',
    'Traffic Control Setup': 'TrafficControlSetup'
}, inplace=True)

# df_valid = df_tt_workzone.copy()
df_valid

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength_TravelTime,Asset Number,SegmentLength_ASSETS,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,38194039865,AP 4-A-Hwy 5-15 • SB • Ledcor Sicim LP • 627.2,3/2/2021 5:31:51 AM,1.00,1.00,1.00,OK,1.000,38194039865,2.693,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914136,38343347632,AP 4-A-13-A • SB • Ledcor Sicim LP • 628.4,3/2/2021 5:31:51 AM,1.00,1.00,1.00,OK,1.000,38343347632,2.790,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914137,77197634355,AP 4-A-Hwy5-13 • NB • Ledcor Sicim LP • 625.9,3/2/2021 5:31:52 AM,1.00,1.00,1.00,OK,1.000,77197634355,2.780,NB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914138,14357322467,AP 4-A-14-A • SB • Ledcor Sicim LP • 631.8,3/2/2021 5:31:52 AM,1.00,1.00,1.00,OK,1.000,14357322467,2.750,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT


In [18]:
# find_data_workzone(df_assets_all_active, file_name='TravelTimesPerWorkZone_AllActive.xlsx')
# find_data_workzone(df_assets_recom, file_name=''TravelTimesPerWorkZone_Recommended.xlsx')

# # with pd.ExcelWriter('TravelTimesPerWorkZone_AllActive.xlsx', engine='xlsxwriter') as writer:
# #     for i, location_id in enumerate(df_assets_all_active['Asset Number']):
# #         df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
# #         df_intersect.to_excel(writer, sheet_name=str(location_id))
# #         if (i+1) % 20 == 0:
# #             print(f'{round((i+1)/len(df_assets_all_active)*100, 2)} %')

# # with pd.ExcelWriter('TravelTimesPerWorkZone_Recommended.xlsx', engine='xlsxwriter') as writer:
# #     for i, location_id in enumerate(df_assets_recom['Asset Number']):
# #         df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
# #         df_intersect.to_excel(writer, sheet_name=str(location_id))
# #         if (i+1) % 20 == 0:
# #             print(f'{round((i+1)/len(df_assets_recom)*100, 2)} %')

## Edit Segment Length


In [19]:
# # Before
# df_valid[(df_valid['SegmentLength_ASSETS'] == 1.0) & (df_valid['SegmentLength_TravelTime'] != 1.0) & (df_valid['AssetNumber'] == 36181764255)]

In [20]:
df_valid['SegmentLength_ASSETS'][
    (df_valid['SegmentLength_ASSETS'] == 1.0) &
    (df_valid['SegmentLength_TravelTime'] != 1.0) &
    (df_valid['AssetNumber'] == 36181764255)] = 2.753
df_valid

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength_TravelTime,Asset Number,SegmentLength_ASSETS,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.363,49831911074,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,38194039865,AP 4-A-Hwy 5-15 • SB • Ledcor Sicim LP • 627.2,3/2/2021 5:31:51 AM,1.00,1.00,1.00,OK,1.000,38194039865,2.693,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914136,38343347632,AP 4-A-13-A • SB • Ledcor Sicim LP • 628.4,3/2/2021 5:31:51 AM,1.00,1.00,1.00,OK,1.000,38343347632,2.790,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914137,77197634355,AP 4-A-Hwy5-13 • NB • Ledcor Sicim LP • 625.9,3/2/2021 5:31:52 AM,1.00,1.00,1.00,OK,1.000,77197634355,2.780,NB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914138,14357322467,AP 4-A-14-A • SB • Ledcor Sicim LP • 631.8,3/2/2021 5:31:52 AM,1.00,1.00,1.00,OK,1.000,14357322467,2.750,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT


In [21]:
# After
df_valid[(df_valid['SegmentLength_ASSETS'] == 1.0) & (df_valid['SegmentLength_TravelTime'] != 1.0) & (df_valid['AssetNumber'] == 36181764255)]

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength_TravelTime,Asset Number,SegmentLength_ASSETS,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup


In [22]:
df_valid.rename(columns = {'SegmentLength_ASSETS': 'SegmentLength'}, inplace=True)
df_valid = df_valid.drop(columns=['Asset Number', 'SegmentLength_TravelTime'])
df_valid

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,38194039865,AP 4-A-Hwy 5-15 • SB • Ledcor Sicim LP • 627.2,3/2/2021 5:31:51 AM,1.00,1.00,1.00,OK,2.693,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914136,38343347632,AP 4-A-13-A • SB • Ledcor Sicim LP • 628.4,3/2/2021 5:31:51 AM,1.00,1.00,1.00,OK,2.790,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914137,77197634355,AP 4-A-Hwy5-13 • NB • Ledcor Sicim LP • 625.9,3/2/2021 5:31:52 AM,1.00,1.00,1.00,OK,2.780,NB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914138,14357322467,AP 4-A-14-A • SB • Ledcor Sicim LP • 631.8,3/2/2021 5:31:52 AM,1.00,1.00,1.00,OK,2.750,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT


## Extract New Columns

### Recalculate Delay Time & State

In [23]:
# Recalculate DelayTime 
df_valid['DelayTime'] = (df_valid.UnderConstructionTime - df_valid.BaselineTime).clip(lower=0).round(4)
# df_valid['CalculatedDelayTime'] = (df_valid.UnderConstructionTime - df_valid.BaselineTime).clip(lower=0).round(4)
df_valid

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,38194039865,AP 4-A-Hwy 5-15 • SB • Ledcor Sicim LP • 627.2,3/2/2021 5:31:51 AM,1.00,1.00,0.00,OK,2.693,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914136,38343347632,AP 4-A-13-A • SB • Ledcor Sicim LP • 628.4,3/2/2021 5:31:51 AM,1.00,1.00,0.00,OK,2.790,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914137,77197634355,AP 4-A-Hwy5-13 • NB • Ledcor Sicim LP • 625.9,3/2/2021 5:31:52 AM,1.00,1.00,0.00,OK,2.780,NB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914138,14357322467,AP 4-A-14-A • SB • Ledcor Sicim LP • 631.8,3/2/2021 5:31:52 AM,1.00,1.00,0.00,OK,2.750,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT


In [24]:
# # Recalculate State
# df_valid['CalculatedState'] = np.where(df_valid['CalculatedDelayTime'] < DELAY_THRESH, 'OK', 'DELAY')
# df_valid

In [25]:
# np.sort(
df_valid = df_valid.drop(
    df_valid[
        (df_valid['AssetNumber'] == 60041619645) &
        (df_valid['UnderConstructionTime'] > 50) &
        (df_valid['BaselineTime'] > 50)
    ].index)
df_valid

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914135,38194039865,AP 4-A-Hwy 5-15 • SB • Ledcor Sicim LP • 627.2,3/2/2021 5:31:51 AM,1.00,1.00,0.00,OK,2.693,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914136,38343347632,AP 4-A-13-A • SB • Ledcor Sicim LP • 628.4,3/2/2021 5:31:51 AM,1.00,1.00,0.00,OK,2.790,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914137,77197634355,AP 4-A-Hwy5-13 • NB • Ledcor Sicim LP • 625.9,3/2/2021 5:31:52 AM,1.00,1.00,0.00,OK,2.780,NB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2914138,14357322467,AP 4-A-14-A • SB • Ledcor Sicim LP • 631.8,3/2/2021 5:31:52 AM,1.00,1.00,0.00,OK,2.750,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT


## Remove 'Low Volume Roadways, No Center Line'

In [26]:
df_valid = df_valid[(df_valid['RoadType'] != 'Low Volume Roadways, No Center Line')].reset_index(drop=True)
df_valid

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup
0,49831911074,Spread 3B,5/3/2022 5:59:55 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
1,49831911074,Spread 3B,5/3/2022 5:57:32 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
2,49831911074,Spread 3B,5/3/2022 5:55:23 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
3,49831911074,Spread 3B,5/3/2022 5:53:29 PM,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
4,49831911074,Spread 3B,5/3/2022 5:51:45 PM,2.23,2.13,0.10,OK,3.360,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913880,38194039865,AP 4-A-Hwy 5-15 • SB • Ledcor Sicim LP • 627.2,3/2/2021 5:31:51 AM,1.00,1.00,0.00,OK,2.693,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2913881,38343347632,AP 4-A-13-A • SB • Ledcor Sicim LP • 628.4,3/2/2021 5:31:51 AM,1.00,1.00,0.00,OK,2.790,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2913882,77197634355,AP 4-A-Hwy5-13 • NB • Ledcor Sicim LP • 625.9,3/2/2021 5:31:52 AM,1.00,1.00,0.00,OK,2.780,NB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT
2913883,14357322467,AP 4-A-14-A • SB • Ledcor Sicim LP • 631.8,3/2/2021 5:31:52 AM,1.00,1.00,0.00,OK,2.750,SB,Spread 4A,High,Multi-Lane Undivided Roadways,SLAT


### From 'CreatedDate'

In [27]:
%%time
df_valid['CreatedDate'] = pd.to_datetime(df_valid['CreatedDate'])

CPU times: user 6min 2s, sys: 251 ms, total: 6min 2s
Wall time: 6min 2s


In [28]:
# Seasons:
# 1: Winter – December, January and February.
# 2: Spring – March, April and May.
# 3: Summer – June, July and August.
# 4: Autumn – September, October and November.
dict_season = {1:'Winter', 2:'Spring', 3:'Summer', 4:'Autumn'}


# DayType:
#     1: Monday (0) & Friday (4)
#     2: OtherWeekday (1,2,3)
#     3: Weekend (5,6)
# dict_day_type = {0:1, 1:2, 2:2, 3:2, 4:1, 5:3, 6:3}
dict_day_type = {0:'Onset/Offset', 1:'Midweek', 2:'Midweek', 3:'Midweek', 4:'Onset/Offset', 5:'Weekend', 6:'Weekend'}


# HourType: 
# Adjust to 18:30 for the border. So 15-18:30 should be peak. For the morning peak 6-9:30
MORNING_PEAK_LOWER = 6*60 
MORNING_PEAK_UPPER = 9*60 + 30
AFTERNOON_PEAK_LOWER = 15*60
AFTERNOON_PEAK_UPPER = 18*60 + 30

# #     Peak (1): 6-9:59, 15-17:59
# #     Off-Peak (2): 10-14:59, 18-5:59
# dict_hour_type = {6:1, 7:1, 8:1,9:1,15:1,16:1,17:1,
#                   10:2,11:2,12:2,13:2,14:2, 18:2,19:2, 20:2, 21:2, 22:2, 23:2, 0:2, 1:2, 2:2, 3:2, 4:2, 5:2}
# #     1: 6-10
# #     2: 10-15
# #     3: 15-18
# #     4: 18-22
# #     5: 22-6
# dict_hour_type = {6:1, 7:1, 8:1,9:1, 
#                   10:2,11:2,12:2,13:2,14:2,
#                   15:3,16:3,17:3,
#                   18:4,19:4,20:4,21:4,
#                   22:5,23:5,0:5,1:5,2:5,3:5,4:5,5:5}

In [29]:
df_valid['CreatedDate:Date'] = df_valid['CreatedDate'].dt.date

# Return the day of the week as an integer, where Monday is 1 and Sunday is 8.
# df_valid['CreatedDate:Weekday'] = df_valid['CreatedDate'].dt.weekday + 1
df_valid['CreatedDate:Weekday'] = df_valid['CreatedDate'].dt.day_name()

# df_valid['CreatedDate:WeekOfYear'] = df_valid['CreatedDate'].dt.week_of_year
# df_valid['CreatedDate:DayOfYear'] = df_valid['CreatedDate'].dt.day_of_year

df_valid['CreatedDate:Year'] = df_valid['CreatedDate'].dt.year
df_valid['CreatedDate:Month'] = df_valid['CreatedDate'].dt.month
# # Between 1 and the number of days in the given month of the given year.
df_valid['CreatedDate:Day'] = df_valid['CreatedDate'].dt.day
df_valid['CreatedDate:Hour'] = df_valid['CreatedDate'].dt.hour
# df_valid['CreatedDate:Minute'] = df_valid['CreatedDate'].dt.minute

df_valid['CreatedDate:MinuteInDay'] = df_valid['CreatedDate'].dt.hour*60 + df_valid['CreatedDate'].dt.minute 

df_valid['CreatedDate:Season'] = (df_valid['CreatedDate'].dt.month%12 // 3 + 1).map(dict_season)
# df_valid['CreatedDate:Season'] = df_valid['CreatedDate'].dt.month%12 // 3 + 1

df_valid['CreatedDate:DayType'] = df_valid['CreatedDate'].dt.weekday.map(dict_day_type)

df_valid['CreatedDate:HourType'] = (
        np.where((MORNING_PEAK_LOWER <= df_valid['CreatedDate:MinuteInDay']) & (df_valid['CreatedDate:MinuteInDay'] <= MORNING_PEAK_UPPER), 'Peak',
                 np.where((AFTERNOON_PEAK_LOWER <= df_valid['CreatedDate:MinuteInDay']) & (df_valid['CreatedDate:MinuteInDay'] <= AFTERNOON_PEAK_UPPER), 'Peak', 'Off-Peak')))
# df_valid['CreatedDate:HourType'] = df_valid['CreatedDate'].dt.hour.map(dict_hour_type)

df_valid

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,...,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,2022-05-03,Tuesday,2022,5,3,17,1079,Spring,Midweek,Peak
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,2022-05-03,Tuesday,2022,5,3,17,1077,Spring,Midweek,Peak
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,2022-05-03,Tuesday,2022,5,3,17,1075,Spring,Midweek,Peak
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,2022-05-03,Tuesday,2022,5,3,17,1073,Spring,Midweek,Peak
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.360,SB,Spread 3B,...,2022-05-03,Tuesday,2022,5,3,17,1071,Spring,Midweek,Peak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913880,38194039865,AP 4-A-Hwy 5-15 • SB • Ledcor Sicim LP • 627.2,2021-03-02 05:31:51,1.00,1.00,0.00,OK,2.693,SB,Spread 4A,...,2021-03-02,Tuesday,2021,3,2,5,331,Spring,Midweek,Off-Peak
2913881,38343347632,AP 4-A-13-A • SB • Ledcor Sicim LP • 628.4,2021-03-02 05:31:51,1.00,1.00,0.00,OK,2.790,SB,Spread 4A,...,2021-03-02,Tuesday,2021,3,2,5,331,Spring,Midweek,Off-Peak
2913882,77197634355,AP 4-A-Hwy5-13 • NB • Ledcor Sicim LP • 625.9,2021-03-02 05:31:52,1.00,1.00,0.00,OK,2.780,NB,Spread 4A,...,2021-03-02,Tuesday,2021,3,2,5,331,Spring,Midweek,Off-Peak
2913883,14357322467,AP 4-A-14-A • SB • Ledcor Sicim LP • 631.8,2021-03-02 05:31:52,1.00,1.00,0.00,OK,2.750,SB,Spread 4A,...,2021-03-02,Tuesday,2021,3,2,5,331,Spring,Midweek,Off-Peak


In [30]:
# df_valid.to_csv(os.path.join(output_root_path, 'AllValidEdited_TravelTimeTaskArchive.csv'), header = True, index=True)
# df_valid.to_excel(os.path.join(output_root_path, 'AllValidEdited_TravelTimeTaskArchive.xlsx'), header = True, index=True)

### From DelayTime

In [31]:
print("MAX(DelayTime) =", df_valid['DelayTime'].max())
df_valid.describe().apply(lambda s: s.apply('{0:.5f}'.format))

MAX(DelayTime) = 106.16


,AssetNumber,UnderConstructionTime,BaselineTime,DelayTime,SegmentLength,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay
count,2913885.00000,2913885.00000,2913845.00000,2913845.00000,2913885.00000,2913885.00000,2913885.00000,2913885.00000,2913885.00000,2913885.00000
mean,54435159370.10207,2.04695,1.96462,0.13960,3.19483,2021.53061,4.26777,15.84742,11.30930,707.21250
std,28639144722.33356,0.77861,0.64006,0.30623,0.97310,0.49906,1.03976,8.94179,6.42464,385.84422
min,10000204812.00000,0.78000,1.00000,0.00000,1.00000,2021.00000,2.00000,1.00000,0.00000,0.00000
25%,25290248946.00000,1.63000,1.63000,0.00000,2.70000,2021.00000,3.00000,9.00000,6.00000,414.00000
50%,53956931309.00000,1.87000,1.80000,0.00000,2.80000,2022.00000,4.00000,15.00000,11.00000,713.00000
75%,83015587391.00000,2.32000,2.17000,0.13000,3.41000,2022.00000,5.00000,24.00000,16.00000,995.00000
max,99994344960.00000,109.03000,189.90000,106.16000,7.92000,2022.00000,6.00000,31.00000,23.00000,1439.00000


In [32]:
LEVEL_COUNT = 4

DELAY_THRESH = df_valid['DelayTime'].max() / 2 

DELAY_MIN_THRESH = 1
DELAY_MAX_THRESH = 5

In [33]:
print(f"LEVEL_COUNT = {LEVEL_COUNT}\n")
print("Class 1: DelayTime =", 0)

if LEVEL_COUNT == 3:
    df_valid['DelayLevel'] = (
        np.where(df_valid['DelayTime'] == 0.0, 1,
                 np.where(df_valid['DelayTime'] > DELAY_THRESH, 3, 2)))
    
    print("Class 2: DelayTime <=", DELAY_THRESH)
    print("Class 3: DelayTime >", DELAY_THRESH)
    print("\DELAY_THRESH =", DELAY_THRESH)

elif LEVEL_COUNT == 4:
    df_valid['DelayLevel'] = (
        np.where(df_valid['DelayTime'] == 0.0, 'None',
                 np.where(df_valid['DelayTime'] > DELAY_MAX_THRESH, 'High',
                          np.where(df_valid['DelayTime'] > DELAY_MIN_THRESH, 'Medium', 'Low'))))
#     df_valid['DelayLevel'] = (
#         np.where(df_valid['DelayTime'] == 0.0, 1,
#                  np.where(df_valid['DelayTime'] > DELAY_MAX_THRESH, 4,
#                           np.where(df_valid['DelayTime'] > DELAY_MIN_THRESH, 3, 2))))
    
    print(f"Class 2: 0 < DelayTime <= {DELAY_MIN_THRESH}")
    print(f"Class 3: {DELAY_MIN_THRESH} < DelayTime <= {DELAY_MAX_THRESH}")
    print("Class 4: DelayTime >", DELAY_MAX_THRESH)

    print("\nDELAY_MIN_THRESH =", DELAY_MIN_THRESH)
    print("DELAY_MAX_THRESH =", DELAY_MAX_THRESH)


# if cond1:
#     exp1
# elif cond2:
#     exp2
# else:
#     exp3
# np.where(cond1, exp1, np.where(cond2, exp2, ...))
df_valid

LEVEL_COUNT = 4

Class 1: DelayTime = 0
Class 2: 0 < DelayTime <= 1
Class 3: 1 < DelayTime <= 5
Class 4: DelayTime > 5

DELAY_MIN_THRESH = 1
DELAY_MAX_THRESH = 5


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,...,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,Tuesday,2022,5,3,17,1079,Spring,Midweek,Peak,Low
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,Tuesday,2022,5,3,17,1077,Spring,Midweek,Peak,Low
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,Tuesday,2022,5,3,17,1075,Spring,Midweek,Peak,Low
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,Tuesday,2022,5,3,17,1073,Spring,Midweek,Peak,Low
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.360,SB,Spread 3B,...,Tuesday,2022,5,3,17,1071,Spring,Midweek,Peak,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913880,38194039865,AP 4-A-Hwy 5-15 • SB • Ledcor Sicim LP • 627.2,2021-03-02 05:31:51,1.00,1.00,0.00,OK,2.693,SB,Spread 4A,...,Tuesday,2021,3,2,5,331,Spring,Midweek,Off-Peak,None
2913881,38343347632,AP 4-A-13-A • SB • Ledcor Sicim LP • 628.4,2021-03-02 05:31:51,1.00,1.00,0.00,OK,2.790,SB,Spread 4A,...,Tuesday,2021,3,2,5,331,Spring,Midweek,Off-Peak,None
2913882,77197634355,AP 4-A-Hwy5-13 • NB • Ledcor Sicim LP • 625.9,2021-03-02 05:31:52,1.00,1.00,0.00,OK,2.780,NB,Spread 4A,...,Tuesday,2021,3,2,5,331,Spring,Midweek,Off-Peak,None
2913883,14357322467,AP 4-A-14-A • SB • Ledcor Sicim LP • 631.8,2021-03-02 05:31:52,1.00,1.00,0.00,OK,2.750,SB,Spread 4A,...,Tuesday,2021,3,2,5,331,Spring,Midweek,Off-Peak,None


In [34]:
# df_valid.loc[df_valid['DelayLevel'] == 'Medium']

# Get Weather Data

In [35]:
df_matched_weather_ids = pd.read_csv(os.path.join(ats_path, 'AssetNumberStationID.csv'), index_col=0)
df_matched_weather_ids.rename(columns = {'Asset Number': 'AssetNumber', 'Station ID': 'Weather:StationID'}, inplace=True)
df_matched_weather_ids

,AssetNumber,Weather:StationID
0,23023696729,706
1,98128214373,706
2,73174024548,706
3,40105790845,706
4,12738620179,706
...,...,...
619,55858030442,794
620,45364364608,794
621,40786939101,794
622,34684428742,49408


In [36]:
df_weather = pd.DataFrame()

# for station_id in df_matched_weather_ids['Weather:StationID'].unique():
for folder_name in os.listdir(weather_path):
    station_id = int(folder_name)
    weather_joined_files = glob.glob(os.path.join(weather_path, f'{station_id}/*.csv'))
    df_weather_station = load_csv_to_df(weather_joined_files)
    
    df_weather_station.insert(loc=0, column='StationID', value=station_id)
    df_weather_station['Date/Time (LST)'] = pd.to_datetime(df_weather_station['Date/Time (LST)'])
    # df_weather_station['Date'] = df_weather_station['Date/Time (LST)'].dt.date
    df_weather_station.insert(loc=df_weather_station.columns.get_loc('Date/Time (LST)')+1, column='Date', value= df_weather_station['Date/Time (LST)'].dt.date)
    df_weather_station.insert(loc=df_weather_station.columns.get_loc('Time (LST)')+1, column='Hour', value= df_weather_station['Date/Time (LST)'].dt.hour)

    df_weather = pd.concat([df_weather, df_weather_station[['StationID', 'Longitude (x)', 'Latitude (y)', 'Date', 'Hour', 'Temp (°C)', 'Precip. Amount (mm)', 'Wind Spd (km/h)']]], axis=0)
    
df_weather.rename(columns = {'Temp (°C)':'Temp',
                             'Precip. Amount (mm)':'Precip',
                             'Wind Spd (km/h)':'WindSpd',
                             'Longitude (x)':'Longitude',
                             'Latitude (y)':'Latitude'},
                  inplace=True)
df_weather.columns = ['Weather:' + str(col) for col in df_weather.columns]

df_weather

,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Date,Weather:Hour,Weather:Temp,Weather:Precip,Weather:WindSpd
0,707,-121.76,49.24,2022-07-01,0,NaN,NaN,NaN
1,707,-121.76,49.24,2022-07-01,1,NaN,NaN,NaN
2,707,-121.76,49.24,2022-07-01,2,NaN,NaN,NaN
3,707,-121.76,49.24,2022-07-01,3,NaN,NaN,NaN
4,707,-121.76,49.24,2022-07-01,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
26299,8214,-119.29,52.13,2020-01-31,19,0.1,1.0,3.0
26300,8214,-119.29,52.13,2020-01-31,20,0.1,0.0,2.0
26301,8214,-119.29,52.13,2020-01-31,21,0.1,0.3,3.0
26302,8214,-119.29,52.13,2020-01-31,22,0.3,0.0,3.0


In [37]:
df_weather.nunique()

Weather:StationID      40
Weather:Longitude      30
Weather:Latitude       26
Weather:Date         1096
Weather:Hour           24
Weather:Temp          842
Weather:Precip        119
Weather:WindSpd        66
dtype: int64

## Add Weather Data to Traffic Time Data

In [38]:
df_tt_workzone = df_valid.copy()
df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_matched_weather_ids, on='AssetNumber')
df_tt_workzone

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,...,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,2022,5,3,17,1079,Spring,Midweek,Peak,Low,1355
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,2022,5,3,17,1077,Spring,Midweek,Peak,Low,1355
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,2022,5,3,17,1075,Spring,Midweek,Peak,Low,1355
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.360,SB,Spread 3B,...,2022,5,3,17,1073,Spring,Midweek,Peak,Low,1355
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.360,SB,Spread 3B,...,2022,5,3,17,1071,Spring,Midweek,Peak,Low,1355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913880,38194039865,AP 4-A-Hwy 5-15 • SB • Ledcor Sicim LP • 627.2,2021-03-02 05:31:51,1.00,1.00,0.00,OK,2.693,SB,Spread 4A,...,2021,3,2,5,331,Spring,Midweek,Off-Peak,None,53423
2913881,38343347632,AP 4-A-13-A • SB • Ledcor Sicim LP • 628.4,2021-03-02 05:31:51,1.00,1.00,0.00,OK,2.790,SB,Spread 4A,...,2021,3,2,5,331,Spring,Midweek,Off-Peak,None,53423
2913882,77197634355,AP 4-A-Hwy5-13 • NB • Ledcor Sicim LP • 625.9,2021-03-02 05:31:52,1.00,1.00,0.00,OK,2.780,NB,Spread 4A,...,2021,3,2,5,331,Spring,Midweek,Off-Peak,None,53423
2913883,14357322467,AP 4-A-14-A • SB • Ledcor Sicim LP • 631.8,2021-03-02 05:31:52,1.00,1.00,0.00,OK,2.750,SB,Spread 4A,...,2021,3,2,5,331,Spring,Midweek,Off-Peak,None,53423


In [39]:
df_tt_workzone = pd.merge(left = df_tt_workzone, right = df_weather,
                          left_on = ['Weather:StationID', 'CreatedDate:Date', 'CreatedDate:Hour'],
                          right_on = ['Weather:StationID', 'Weather:Date', 'Weather:Hour'])
df_tt_workzone = df_tt_workzone.drop(columns=['Weather:Date', 'Weather:Hour'])

df_tt_workzone

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,...,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,...,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913880,61630079816,AP 3-C-58-B • NB • Ledcor Sicim LP • 561.7,2021-03-03 02:45:09,1.62,1.60,0.02,OK,2.81,NB,Spread 3C,...,Spring,Midweek,Off-Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN
2913881,61630079816,AP 3-C-58-B • NB • Ledcor Sicim LP • 561.7,2021-03-03 02:50:09,1.62,1.60,0.02,OK,2.81,NB,Spread 3C,...,Spring,Midweek,Off-Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN
2913882,61630079816,AP 3-C-58-B • NB • Ledcor Sicim LP • 561.7,2021-03-03 02:55:09,1.62,1.60,0.02,OK,2.81,NB,Spread 3C,...,Spring,Midweek,Off-Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN
2913883,61630079816,AP 3-C-58-B • NB • Ledcor Sicim LP • 561.7,2021-03-03 03:00:09,1.62,1.60,0.02,OK,2.81,NB,Spread 3C,...,Spring,Midweek,Off-Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN


### Checking Weather data

In [40]:
#  df_weather[(df_weather['Weather:Date'] == df_tt_workzone.iloc[0]['CreatedDate:Date']) &
#             (df_weather['Weather:Hour'] == 20) & 
#             (df_weather['Weather:StationID'] == 1355)]
    
# # df_tt_workzone.iloc[0]['CreatedDate:Date']

In [41]:
# df_matched_weather_ids[df_matched_weather_ids['Weather:StationID'] == 1355]

In [42]:
# df_weather[df_weather['Weather:Temp'].isnull()]

In [43]:
# def find_weather_info(tt_entry, dict_tt_help_idx):
#     station_id = dict_matched_ids[tt_entry[dict_tt_help_idx['AssetNumber']]] 
#     weather_data = dict_weather[station_id]

#     weather_record = weather_data[(weather_data['Date/Time (LST)'].dt.date == tt_entry[dict_tt_help_idx['CreatedDate']].date())
#                                   & (weather_data['Date/Time (LST)'].dt.hour == tt_entry[dict_tt_help_idx['CreatedDate']].hour)]

#     return weather_record


# def dict_funct():
#     temp = df_tt_workzone.to_dict('records')
#     tt_idx = 0

#     for tt_entry in temp:
#         if tt_entry['AssetNumber'] in dict_matched_ids:
#             weather_record = find_weather_info(tt_entry)

#             df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Temp']] = weather_record['Temp (°C)'].values[0]
#             df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Precip']] = weather_record['Precip. Amount (mm)'].values[0]
#             df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:WindSpd']] = weather_record['Wind Spd (km/h)'].values[0]

#         if tt_idx % 10000 == 0:
#             print(f'{(tt_idx+1)/len(df_tt_workzone)*100:.2f}%')
#         tt_idx += 1

#     return df_tt_workzone


# def add_weather_data(df_tt, start, stop):
#     print(start, stop)
#     df_tt = df_tt.reset_index()

#     df_tt['Weather:Temp'] = ''
#     df_tt['Weather:Precip'] = ''
#     df_tt['Weather:WindSpd'] = ''

#     dict_tt_help_idx = {'Weather:Temp': df_tt.columns.get_loc('Weather:Temp'),
#                         'Weather:Precip': df_tt.columns.get_loc('Weather:Precip'),
#                         'Weather:WindSpd': df_tt.columns.get_loc('Weather:WindSpd'),
#                         'CreatedDate': df_tt.columns.get_loc('CreatedDate'),
#                         'AssetNumber': df_tt.columns.get_loc('AssetNumber')}
    
#     df_tt_workzone = dict_funct(df_tt, dict_tt_help_idx, start, stop)
#     df_tt_workzone.to_csv(os.path.join(output_root_path, f'processed_travel_time_{start}-{stop}.csv'))  
#     return df_tt_workzone

In [44]:
del df_weather

# Get Counters Data

In [45]:
df_matched_counters_ids = pd.read_excel(os.path.join(counter_path, 'NearestCountStationsToAssets.xlsx'))[['Asset_Numb', 'prefixSiteNo']]
df_matched_counters_ids.rename(columns = {'Asset_Numb': 'AssetNumber', 'prefixSiteNo': 'Traffic:SiteNo'}, inplace=True)
df_matched_counters_ids

,AssetNumber,Traffic:SiteNo
0,23023696729,17-044EW
1,98128214373,17-044EW
2,25033704574,23-009NS
3,71808680042,23-010NS
4,27743462715,23-010NS
...,...,...
619,12066281555,21-011NS
620,62134790681,23-010NS
621,33379829945,23-010NS
622,21272146931,P-27-1EW


In [46]:
def clean_traffic_data_perm(df):
    if df.shape[1] == 35:
        df = df.drop(columns=['Unnamed: 7', 'Unnamed: 31', 'Unnamed: 34'])
    elif df.shape[1] == 34: 
        df = df.drop(columns=[ 'Unnamed: 30', 'Unnamed: 33'])
    elif df.shape[1] == 32:
        df = df.drop(columns=['Unnamed: 28', 'Unnamed: 31'])
    elif df.shape[1] == 30:
        df = df.drop(columns=['Unnamed: 26', 'Unnamed: 29'])
    
    df = df.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 9', 'Unnamed: 17'])
    df = df.dropna(how='all', axis=1)
    df = df.dropna(axis=0)
    
    df.rename(columns = {'Unnamed: 6':'Date'}, inplace = True)
    df = df.reset_index(drop=True)
    return df


def clean_traffic_data_temp(df):
    df = raw_traffic_site.copy()
    df = df.iloc[5:29, [1,3,6,9,12,15,18,21]]
#     df[['County:', 'N/A', 'Unnamed: 6', 'Unnamed: 9', 'Unnamed: 12', 'Unnamed: 15', 'Unnamed: 18', 'Unnamed: 21']].iloc[5:29]
    df.rename(columns = {'County:':'Hour', 'N/A':'Unnamed: 3'}, inplace = True)
    df['Hour'] =  pd.to_datetime(df['Hour']).dt.hour
    df = df.set_index('Hour')
    df['Road'] = df.mean(axis=1)
    return df[['Road']].reset_index()

In [47]:
ignore_stations = ['21-012NS', '23-005EW', 'W-23-2EW']

In [48]:
df_traffic = {'Temp': pd.DataFrame(), 'Perm': pd.DataFrame()}

# for site_no in df_matched_counters_ids['prefixSiteNo'].unique():
for site_no in os.listdir(os.path.join(counter_path, f'TrafficInfo')):
    if site_no in ignore_stations:
        continue
        
    site_type = 'DV01' if site_no.startswith('P') else 'DV03S' 
    
    counter_joined_files = glob.glob(os.path.join(counter_path, f'TrafficInfo/{site_no}/{site_type}*.xls'))
    for f in counter_joined_files:
        raw_traffic_site = pd.read_excel(f, header=5, thousands=',')
    
    df_traffic_site = pd.DataFrame()
#     df_traffic_site['Year'] = int(counter_joined_files[0][-8:-4])
# #     df_traffic_site['CreatedDate'].year - 1

    if site_no.startswith('P'):
#         print("\n****BINGO***")
#         print(site_no, raw_traffic_site.shape, '\n')
        clean_traffic_site = clean_traffic_data_perm(raw_traffic_site)
        temp_datetime = pd.to_datetime(clean_traffic_site['Date'])
        df_traffic_site['AADT'] = clean_traffic_site['Road'].astype(float)
        df_traffic_site['BaseYear'] = temp_datetime.dt.year
        df_traffic_site['Month'] = temp_datetime.dt.month
        df_traffic_site['Day'] = temp_datetime.dt.day
        df_traffic_site['Hour'] = temp_datetime.dt.hour
        df_traffic_site.insert(loc=0, column='SiteNo', value=site_no)
        df_traffic_site.insert(loc=1, column='IsTemporary', value=False)
        df_traffic_site.insert(loc=len(df_traffic_site.columns), column='CreatedDate', value=pd.to_datetime(raw_traffic_site.iloc[-1][3]))
        df_traffic['Perm'] = pd.concat([df_traffic['Perm'], df_traffic_site], axis=0)
        
    else: #Temp stations
#         print(site_no, raw_traffic_site.shape)
        clean_traffic_site = clean_traffic_data_temp(raw_traffic_site)
        
        temp_datetime = pd.to_datetime(counter_joined_files[0][-8:-4]).date()   # YEAR in datetime format    
#         USE HOURLY FIELD (ROAD) OF DATA
        df_traffic_site['AADT'] = clean_traffic_site['Road']
        df_traffic_site['BaseYear'] = temp_datetime.year
        df_traffic_site['Month'] = temp_datetime.month
        df_traffic_site['Day'] = temp_datetime.day
        df_traffic_site['Hour'] = clean_traffic_site['Hour']

# #         JUST USE ONE FIELD (ROAD AADT) OF DATA
#         df_traffic_site['AADT'] = [float(raw_traffic_site.iloc[-1][13])]
#         df_traffic_site['BaseYear'] = [temp_datetime.year]
#         df_traffic_site['Month'] = [temp_datetime.month]
#         df_traffic_site['Day'] = [temp_datetime.day]

        df_traffic_site.insert(loc=0, column='SiteNo', value=site_no)
        df_traffic_site.insert(loc=1, column='IsTemporary', value=True)
        df_traffic_site.insert(loc=len(df_traffic_site.columns), column='CreatedDate', value=pd.to_datetime(raw_traffic_site.iloc[-1][1]).date())
#         df_traffic_site['CreatedDate'] = [pd.to_datetime(raw_traffic_site.iloc[-1][1]).date()]
        
        df_traffic['Temp'] = pd.concat([df_traffic['Temp'], df_traffic_site], axis=0)
#         break
        
for site_type in df_traffic:
    df_traffic[site_type].columns = ['Traffic:' + str(col) for col in df_traffic[site_type].columns]
    df_traffic[site_type] = df_traffic[site_type].reset_index(drop=True)

df_traffic['Perm']

,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:Hour,Traffic:CreatedDate
0,P-17-1EW,False,9251.0,2021,1,1,0,2022-03-08
1,P-17-1EW,False,12042.0,2021,1,2,0,2022-03-08
2,P-17-1EW,False,13890.0,2021,1,3,0,2022-03-08
3,P-17-1EW,False,10507.0,2021,1,4,0,2022-03-08
4,P-17-1EW,False,10812.0,2021,1,5,0,2022-03-08
...,...,...,...,...,...,...,...,...
1825,P-23-3NS,False,1668.0,2021,12,27,0,2022-03-09
1826,P-23-3NS,False,1751.0,2021,12,28,0,2022-03-09
1827,P-23-3NS,False,1748.0,2021,12,29,0,2022-03-09
1828,P-23-3NS,False,1751.0,2021,12,30,0,2022-03-09


In [49]:
df_traffic['Temp']

,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:Hour,Traffic:CreatedDate
0,27-001E,True,110.50,2013,1,1,0,2014-01-20
1,27-001E,True,87.25,2013,1,1,1,2014-01-20
2,27-001E,True,51.75,2013,1,1,2,2014-01-20
3,27-001E,True,61.50,2013,1,1,3,2014-01-20
4,27-001E,True,87.75,2013,1,1,4,2014-01-20
...,...,...,...,...,...,...,...,...
619,27-033W,True,43.00,2013,1,1,19,2014-01-20
620,27-033W,True,31.75,2013,1,1,20,2014-01-20
621,27-033W,True,23.00,2013,1,1,21,2014-01-20
622,27-033W,True,12.00,2013,1,1,22,2014-01-20


## Add Counters Data to Traffic Time Data

In [50]:
pd.set_option('display.max_columns', None)

In [51]:
GROWTH_RATE = 0.01

In [52]:
df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_matched_counters_ids, on='AssetNumber')
# df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_matched_counters_ids, left_on='AssetNumber', right_on='AssetNumber')

df_tt_workzone

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1079,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1077,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1075,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1073,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1071,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913880,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1120,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W
2913881,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1125,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W
2913882,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1130,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W
2913883,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:51:07,1.37,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1131,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W


In [53]:
def find_diff(a, b):
    return list(set(a).difference(b))
    

# set(df_tt_workzone['Traffic:SiteNo'].unique()).difference(set(os.listdir(os.path.join(counter_path, f'TrafficInfo'))))
missing_site = find_diff(df_tt_workzone['Traffic:SiteNo'].unique(), os.listdir(os.path.join(counter_path, f'TrafficInfo')))

missing_data_count = len(df_tt_workzone[df_tt_workzone['Traffic:SiteNo'].isin(ignore_stations + missing_site)])
missing_data_count

157460

In [54]:
# tt = pd.merge(left = df_tt_workzone, right = df_traffic['Temp'], on = 'Traffic:SiteNo', how='left')
# tt = pd.merge(left = df_tt_workzone, right = df_traffic['Temp'], on = 'Traffic:SiteNo', how='inner')
join_temp_traffic = df_tt_workzone.reset_index().merge(df_traffic['Temp'], how="inner",
                                                       left_on = ['Traffic:SiteNo', 'CreatedDate:Hour'],
                                                       right_on = ['Traffic:SiteNo', 'Traffic:Hour']
                                                      ).set_index('index')
join_temp_traffic

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:Hour,Traffic:CreatedDate
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1079,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017,1,1,17,2018-01-25
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1077,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017,1,1,17,2018-01-25
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1075,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017,1,1,17,2018-01-25
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1073,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017,1,1,17,2018-01-25
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1071,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017,1,1,17,2018-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913828,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 16:40:10,1.35,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,16,1000,Spring,Weekend,Peak,None,706,-121.76,49.24,15.2,0.0,8.0,17-015W,True,835.333333,2013,1,1,16,2014-01-20
2913829,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 16:41:05,1.33,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,16,1001,Spring,Weekend,Peak,None,706,-121.76,49.24,15.2,0.0,8.0,17-015W,True,835.333333,2013,1,1,16,2014-01-20
2913830,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 16:45:10,1.33,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,16,1005,Spring,Weekend,Peak,None,706,-121.76,49.24,15.2,0.0,8.0,17-015W,True,835.333333,2013,1,1,16,2014-01-20


In [55]:
# tp = pd.merge(left = df_tt_workzone, right = df_traffic['Perm'], 
#          left_on = ['Traffic:SiteNo', 'CreatedDate:Month', 'CreatedDate:Day'],
#          right_on = ['Traffic:SiteNo', 'Traffic:Month', 'Traffic:Day'],
#              how='inner')
join_perm_traffic = df_tt_workzone.reset_index().merge(df_traffic['Perm'], how="inner",
                                                       left_on = ['Traffic:SiteNo', 'CreatedDate:Month', 'CreatedDate:Day'],
                                                       right_on = ['Traffic:SiteNo', 'Traffic:Month', 'Traffic:Day']
                                                      ).set_index('index')
join_perm_traffic

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:Hour,Traffic:CreatedDate
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
236149,72495793531,Reactivation,2022-05-03 15:44:24,1.28,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,944,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.0,2021,5,3,0,2022-03-09
236150,72495793531,Reactivation,2022-05-03 15:42:05,1.28,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,942,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.0,2021,5,3,0,2022-03-09
236151,72495793531,Reactivation,2022-05-03 15:39:54,1.23,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,939,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.0,2021,5,3,0,2022-03-09
236152,72495793531,Reactivation,2022-05-03 15:37:36,1.28,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,937,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.0,2021,5,3,0,2022-03-09
236153,72495793531,Reactivation,2022-05-03 15:35:29,1.28,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,935,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.0,2021,5,3,0,2022-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2240480,21382867516,Spread 5A,2022-05-12 08:18:39,2.30,2.55,0.0,OK,2.54,SB,Spread 5A,High,Multi-Lane Divided,Lane Closure,2022-05-12,Thursday,2022,5,12,8,498,Spring,Midweek,Peak,None,1022,-120.80,50.11,NaN,NaN,NaN,P-27-1EW,False,1973.0,2021,5,12,0,2022-03-08
2240481,21382867516,Spread 5A,2022-05-12 08:16:37,2.25,2.55,0.0,OK,2.54,SB,Spread 5A,High,Multi-Lane Divided,Lane Closure,2022-05-12,Thursday,2022,5,12,8,496,Spring,Midweek,Peak,None,1022,-120.80,50.11,NaN,NaN,NaN,P-27-1EW,False,1973.0,2021,5,12,0,2022-03-08
2240482,21382867516,Spread 5A,2022-05-12 08:14:32,2.23,2.55,0.0,OK,2.54,SB,Spread 5A,High,Multi-Lane Divided,Lane Closure,2022-05-12,Thursday,2022,5,12,8,494,Spring,Midweek,Peak,None,1022,-120.80,50.11,NaN,NaN,NaN,P-27-1EW,False,1973.0,2021,5,12,0,2022-03-08


In [56]:
df_tt_workzone_join = pd.concat([join_perm_traffic, join_temp_traffic], axis=0)
df_tt_workzone_join['Traffic:AADT'] * (1 + (GROWTH_RATE * (df_tt_workzone_join['CreatedDate:Year'] - df_tt_workzone_join['Traffic:BaseYear'])))
df_tt_workzone_join = df_tt_workzone_join.drop(columns=['Traffic:Month', 'Traffic:Day', 'Traffic:Hour', 'Traffic:CreatedDate'])
df_tt_workzone_join['Traffic:CalculatedAADT'] = df_tt_workzone_join['Traffic:AADT'] * (1 + (GROWTH_RATE * (df_tt_workzone_join['CreatedDate:Year'] - df_tt_workzone_join['Traffic:BaseYear'])))
df_tt_workzone_join

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
236149,72495793531,Reactivation,2022-05-03 15:44:24,1.28,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,944,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.000000,2021,2881.53
236150,72495793531,Reactivation,2022-05-03 15:42:05,1.28,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,942,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.000000,2021,2881.53
236151,72495793531,Reactivation,2022-05-03 15:39:54,1.23,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,939,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.000000,2021,2881.53
236152,72495793531,Reactivation,2022-05-03 15:37:36,1.28,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,937,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.000000,2021,2881.53
236153,72495793531,Reactivation,2022-05-03 15:35:29,1.28,1.30,0.0,OK,1.81,NB,Reactivation,Low,"Two-Lane, Two-Way Roadways (>70 km/h)",Brief Duration,2022-05-03,Tuesday,2022,5,3,15,935,Spring,Midweek,Peak,None,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2853.000000,2021,2881.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913828,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 16:40:10,1.35,1.48,0.0,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,16,1000,Spring,Weekend,Peak,None,706,-121.76,49.24,15.2,0.0,8.0,17-015W,True,835.333333,2013,902.16
2913829,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 16:41:05,1.33,1.48,0.0,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,16,1001,Spring,Weekend,Peak,None,706,-121.76,49.24,15.2,0.0,8.0,17-015W,True,835.333333,2013,902.16
2913830,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 16:45:10,1.33,1.48,0.0,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,16,1005,Spring,Weekend,Peak,None,706,-121.76,49.24,15.2,0.0,8.0,17-015W,True,835.333333,2013,902.16


In [57]:
incomplete_idx = find_diff(df_tt_workzone.index.values, df_tt_workzone_join.index.values)
incomplete_entries = df_tt_workzone[df_tt_workzone.index.isin(incomplete_idx)]
print('# incomplete_entries:', len(incomplete_entries))

df_tt_workzone = pd.concat([df_tt_workzone_join, incomplete_entries], axis=0).sort_index()
# df_tt_workzone = df_tt_workzone.drop(columns='AllTasksIndex')
df_tt_workzone

# incomplete_entries: 386761


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1079,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1077,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1075,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1073,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1071,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913880,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1120,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72
2913881,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1125,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72
2913882,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1130,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72
2913883,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:51:07,1.37,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1131,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72


In [58]:
# pd.merge(left = tp[:count], right = tt[:count], on = ['Traffic:SiteNo','Traffic:IsTemporary'], how='inner')
# pd.merge(left = tp[:count], right = df_traffic['Temp'], on = 'Traffic:SiteNo', how='inner')

In [59]:
del df_traffic
del join_perm_traffic
del join_temp_traffic
del incomplete_entries
del df_tt_workzone_join

# Get Speed Data

In [60]:
df_assets_speed = pd.read_csv(os.path.join(ats_path, 'AssetsNumberMaxSpeedALL.csv')).drop(columns=['Unnamed: 0'])
df_assets_speed.rename(columns = {'Asset_Numb': 'AssetNumber', 'maxspeed': 'MaxSpeed'}, inplace=True)
df_assets_speed

,AssetNumber,name,fclass,oneway,MaxSpeed
0,10000204812,Southern Yellowhead Highway,trunk,B,100.0
1,10201541400,Coquihalla Highway,motorway,F,120.0
2,10266236577,Southern Yellowhead Highway,trunk,B,100.0
3,11042076244,Yellowhead Highway,trunk,B,100.0
4,11181395533,Southern Yellowhead Highway,trunk,B,100.0
...,...,...,...,...,...
589,12285083600,NaN,NaN,NaN,NaN
590,55324137435,NaN,NaN,NaN,NaN
591,19316607964,NaN,NaN,NaN,NaN
592,61187226602,NaN,NaN,NaN,NaN


## Add Speed Data to Traffic Time Data

In [61]:
df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_assets_speed[['AssetNumber', 'MaxSpeed']], on='AssetNumber')
df_tt_workzone

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT,MaxSpeed
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1079,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1077,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1075,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1073,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1071,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815816,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1120,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN
2815817,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1125,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN
2815818,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1130,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN
2815819,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:51:07,1.37,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1131,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN


In [62]:
del df_assets_speed
gc.collect()

21

# Convert to Categorical type

In [63]:
len(df_valid[(df_valid['TrafficControlSetup'] == 'Brief Duration')])

485005

In [64]:
column_categories = {
    'TravelDirect':['SB', 'NB'],
    'Spread': ['Reactivation', 'Spread 3B', 'Spread 3C', 'Spread 4A', 'Spread 4B', 'Spread 5A', 'Spread 5B'],
    'TrafficImpact':['Low', 'Moderate', 'High'],
    'RoadType':['Two-Lane, Two-Way Roadways (>70 km/h)', 'Multi-Lane Undivided Roadways', 'Multi-Lane Divided', 'Low Volume Roadways, No Center Line'],
    'TrafficControlSetup':['SLAT', 'Brief Duration', 'Lane Closure'],
    'CreatedDate:Season': ['Spring', 'Summer', 'Autumn', 'Winter'],
    'CreatedDate:DayType': ['Onset/Offset', 'Midweek', 'Weekend'],
    'CreatedDate:HourType': ['Peak', 'Off-Peak'],
    'DelayLevel': ['None', 'Low', 'Medium', 'High']}

In [65]:
for col_name, col_categories in column_categories.items():
    temp = pd.Categorical(df_tt_workzone[col_name], categories=col_categories)
    df_tt_workzone[f'{col_name}_Code'] = temp.codes
    #     df_valid[f'{col_name}_Code'] = df_valid[col_name].cat.codes
#     .cat.set_categories(['None', 'Low', 'Medium', 'High'])
#     df_valid[f'{col_name}_Code'] = df_valid[col_name].astype('category').cat.codes
    
df_tt_workzone

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,TrafficImpact,RoadType,TrafficControlSetup,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT,MaxSpeed,TravelDirect_Code,Spread_Code,TrafficImpact_Code,RoadType_Code,TrafficControlSetup_Code,CreatedDate:Season_Code,CreatedDate:DayType_Code,CreatedDate:HourType_Code,DelayLevel_Code
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1079,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0,0,1,2,0,0,0,1,0,1
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1077,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0,0,1,2,0,0,0,1,0,1
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1075,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0,0,1,2,0,0,0,1,0,1
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1073,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0,0,1,2,0,0,0,1,0,1
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,High,"Two-Lane, Two-Way Roadways (>70 km/h)",SLAT,2022-05-03,Tuesday,2022,5,3,17,1071,Spring,Midweek,Peak,Low,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0,0,1,2,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815816,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1120,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN,1,6,2,2,2,0,2,1,0
2815817,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1125,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN,1,6,2,2,2,0,2,1,0
2815818,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1130,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN,1,6,2,2,2,0,2,1,0
2815819,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:51:07,1.37,1.48,0.00,OK,2.45,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,2021-05-01,Saturday,2021,5,1,18,1131,Spring,Weekend,Off-Peak,None,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN,1,6,2,2,2,0,2,1,0


In [66]:
df_tt_workzone.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,AssetNumber,UnderConstructionTime,BaselineTime,DelayTime,SegmentLength,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,Weather:StationID,Weather:Longitude,Weather:Latitude,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT,MaxSpeed,TravelDirect_Code,Spread_Code,TrafficImpact_Code,RoadType_Code,TrafficControlSetup_Code,CreatedDate:Season_Code,CreatedDate:DayType_Code,CreatedDate:HourType_Code,DelayLevel_Code
count,2815821.00000,2815821.00000,2815786.00000,2815786.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,1183708.00000,1053363.00000,1183708.00000,2436810.00000,2436810.00000,2436810.00000,2347522.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000,2815821.00000
mean,54762076691.73260,2.05512,1.97355,0.13907,3.21350,2021.53240,4.26086,15.84867,11.30817,707.14634,12500.00312,-119.89548,51.60290,8.84338,0.10346,7.67306,302.45714,2015.50660,312.89329,101.67137,0.50197,2.87075,1.61466,0.91190,0.63475,0.13579,0.87406,0.67967,0.51182
std,28621142084.03205,0.78477,0.64731,0.30557,0.98290,0.49895,1.03839,8.90502,6.43079,386.21294,20121.90416,1.10183,1.36682,6.33744,0.43491,5.43895,544.93765,3.17090,546.80415,7.34419,0.50000,2.08544,0.75988,0.74699,0.84287,0.35376,0.70011,0.46661,0.54594
min,10000204812.00000,0.85000,1.00000,0.00000,1.00000,2021.00000,2.00000,1.00000,0.00000,0.00000,706.00000,-121.76000,49.24000,-15.40000,0.00000,0.00000,7.50000,2009.00000,8.17500,70.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,26620220646.00000,1.62000,1.63000,0.00000,2.71000,2021.00000,3.00000,9.00000,6.00000,412.00000,1237.00000,-120.18000,51.29000,5.60000,0.00000,3.00000,61.33333,2016.00000,64.40000,100.00000,0.00000,1.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
50%,53956931309.00000,1.88000,1.80000,0.00000,2.85000,2022.00000,4.00000,15.00000,11.00000,714.00000,1355.00000,-119.47000,52.13000,9.40000,0.00000,7.00000,135.00000,2017.00000,147.34000,100.00000,1.00000,2.00000,2.00000,1.00000,0.00000,0.00000,1.00000,1.00000,0.00000
75%,84067785913.00000,2.32000,2.18000,0.13000,3.51000,2022.00000,5.00000,24.00000,16.00000,996.00000,10223.00000,-119.29000,52.72000,12.80000,0.00000,11.00000,347.66667,2017.00000,365.05000,100.00000,1.00000,4.00000,2.00000,1.00000,1.00000,0.00000,1.00000,1.00000,1.00000
max,99994344960.00000,109.03000,189.90000,106.16000,7.92000,2022.00000,6.00000,31.00000,23.00000,1439.00000,53423.00000,-118.03000,52.93000,29.90000,7.70000,41.00000,3926.00000,2021.00000,3926.00000,120.00000,1.00000,6.00000,2.00000,2.00000,2.00000,3.00000,2.00000,1.00000,3.00000


In [67]:
# df_tt_workzone.loc[df_tt_workzone['DelayLevel'] == 'Medium']

In [68]:
# df_tt_workzone.loc[df_tt_workzone['CreatedDate:Month'] == 2]

# Exploratory Data Analysis

## General Info about dataframe

In [69]:
# df_tt_workzone.describe().to_csv(os.path.join(output_root_path, 'data_descriptive_statistics_PYTHON.csv'))
# df_tt_workzone.describe().apply(lambda s: s.apply('{0:.5f}'.format))
# # df_tt_workzone.describe(include='all')
# # df_tt_workzone.describe(include='object')

In [70]:
df_tt_workzone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2815821 entries, 0 to 2815820
Data columns (total 45 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   AssetNumber                int64         
 1   AssetIdentifier            object        
 2   CreatedDate                datetime64[ns]
 3   UnderConstructionTime      float64       
 4   BaselineTime               float64       
 5   DelayTime                  float64       
 6   State                      object        
 7   SegmentLength              float64       
 8   TravelDirect               object        
 9   Spread                     object        
 10  TrafficImpact              object        
 11  RoadType                   object        
 12  TrafficControlSetup        object        
 13  CreatedDate:Date           object        
 14  CreatedDate:Weekday        object        
 15  CreatedDate:Year           int64         
 16  CreatedDate:Month          int64    

In [71]:
df_tt_workzone.isnull().sum()

AssetNumber                        0
AssetIdentifier                    0
CreatedDate                        0
UnderConstructionTime              0
BaselineTime                      35
DelayTime                         35
State                            161
SegmentLength                      0
TravelDirect                       0
Spread                             0
TrafficImpact                      0
RoadType                           0
TrafficControlSetup                0
CreatedDate:Date                   0
CreatedDate:Weekday                0
CreatedDate:Year                   0
CreatedDate:Month                  0
CreatedDate:Day                    0
CreatedDate:Hour                   0
CreatedDate:MinuteInDay            0
CreatedDate:Season                 0
CreatedDate:DayType                0
CreatedDate:HourType               0
DelayLevel                         0
Weather:StationID                  0
Weather:Longitude                  0
Weather:Latitude                   0
W

# Export

In [72]:
del df_valid
gc.collect()

42

In [73]:
%%time
# temp = df_tt_workzone.copy()
df_tt_workzone.columns = df_tt_workzone.columns.str.replace("[#,@,&,:,-,.,',[,]", '_')
pyreadstat.write_sav(df_tt_workzone, 'TravelTime_Valid_Recom_Weather_Traffic.sav')
# del temp

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


CPU times: user 1min 35s, sys: 14.5 s, total: 1min 49s
Wall time: 1min 49s


In [74]:
df_tt_workzone.to_csv(os.path.join(output_root_path, 'TravelTime_Valid_Recom_Weather_Traffic.csv'))
# df_tt_workzone.to_csv(os.path.join(output_root_path, 'TravelTime_Valid_Recom_Weather_Traffic+.csv'), header = True, index=True)

In [75]:
def find_delay_diff():
    diff_delay = df_valid[df_valid['DelayTime'] != df_valid['CalculatedDelayTime']]
    diff_delay.to_csv(os.path.join(output_root_path, 'wrong_DelayTime.csv'), header = True, index=True)
    return diff_delay

In [76]:
def find_state_diff():
    diff_state = df_valid[df_valid['State'] != df_valid['CalculatedState']]
    diff_state.to_csv(os.path.join(output_root_path, 'wrong_State.csv'), header = True, index=True)
    return diff_state

## Plot

In [77]:
import seaborn as sns
import matplotlib.pyplot as plt
# from PIL import Image

%matplotlib inline
sns.set_style("darkgrid")
plt.style.use('ggplot')

STEP_SIZE = 0.5

In [78]:
def draw_time_plot(loc, loc_id, time_type, plt_name):
    fig, ax = plt.subplots()

    # loc.plot.scatter(x='CreatedDate:DayName', y=['UnderConstructionTime', 'BaselineTime'])
    # loc.plot.scatter(x='CreatedDate:DayName', y='UnderConstructionTime')
#     plt.scatter(x=loc[f'CreatedDate:{time_type}'], y=loc['UnderConstructionTime'], color='purple', marker='o', label='UnderConstructionTime (After)')
#     plt.scatter(x=loc[f'CreatedDate:{time_type}'], y=loc['BaselineTime'], color='orange', marker='h', label='BaselineTime (Before)')

    x, y  = [], []
    
    if time_type == 'Hour':
        for hour in range(24):
            loc_hour = loc.loc[loc[f'CreatedDate:{time_type}'] == hour]
            x.append(hour)
            y.append(loc_hour['UnderConstructionTime'].mean())
    
    elif time_type == 'DayName':
         for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
            loc_day = loc.loc[loc[f'CreatedDate:{time_type}'] == day]
            x.append(day)
            y.append(loc_day['UnderConstructionTime'].mean())
    
    temp = loc['BaselineTime'].mean()
    plt.plot(x, y, color='purple', marker='o', label='UnderConstructionTime (After)')
    plt.plot(x, [temp for i in range(len(y))], color='orange', marker='h', label='BaselineTime (Before)')
    
#     start, end = ax.get_ylim()
#     plt.yticks(np.arange(np.round(start), np.round(end)+STEP_SIZE, step=STEP_SIZE))
    if time_type == 'Hour':
        plt.xticks(np.arange(0, 24, step=1))
    
    fig.set_size_inches(14, 10)
    ax.legend()
    ax.grid(True)
    plt.title(f'Travel Time for AssetID {loc_id} ({time_type})')
    plt.xlabel(time_type)
    plt.ylabel('TravelTime')
    plt.tight_layout()
    plt.savefig(f'{output_root_path}/{plt_name}.png')
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()

### Draw plot for selected asset ids

In [79]:
locations_id = [80002045894, 50253718586, 44853071770, 12738620179, 37528363723, 43805438913, 68881926366, 36674989230, 34592545857, 22685608926]

In [80]:
# for location_id in locations_id:
#     for time_type in ['DayName', 'Hour']:
#         location = df_valid.loc[df_valid['AssetNumber'] == location_id]
#         draw_time_plot(location, location_id, time_type, plt_name=f'{location_id}_{time_type}')

# Zip outputs

In [81]:
!zip ouput.zip ../working/*

  adding: ../working/TravelTime_Valid_Recom_Weather_Traffic.csv (deflated 96%)
  adding: ../working/TravelTime_Valid_Recom_Weather_Traffic.sav (deflated 98%)
  adding: ../working/__notebook__.ipynb (deflated 91%)
